In [22]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time


from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

## Building the 1D-CNN with residual connections.

In [23]:
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        # First conv layer (uses passed in in_channels, NOT hard-coded)
        self.conv1 = nn.Conv1d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=stride,
            padding=1
        )
        self.bn1 = nn.BatchNorm1d(out_channels)

        # Second conv layer
        self.conv2 = nn.Conv1d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.bn2 = nn.BatchNorm1d(out_channels)

        # Downsample (skip connection) if needed
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv1d(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=1,
                    stride=stride
                ),
                nn.BatchNorm1d(out_channels)
            )
        else:
            self.downsample = None

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Apply skip path if needed
        if self.downsample is not None:
            identity = self.downsample(identity)

        out = out + identity
        out = self.relu(out)
        return out


In [24]:
class ResNet1D(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        
        # Initial convolution "stem"
        self.conv1 = nn.Conv1d(
            in_channels=n_channels,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=3
        )
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # Residual stages
        self.layer1 = self._make_layer(64, 64, num_blocks=2, stride=1)
        self.layer2 = self._make_layer(64, 128, num_blocks=2, stride=2)
        self.layer3 = self._make_layer(128, 256, num_blocks=2, stride=2)
        self.layer4 = self._make_layer(256, 512, num_blocks=2, stride=2)
        
        # Global average pooling over time dimension
        self.global_pool = nn.AdaptiveAvgPool1d(1)  # output: (batch, channels, 1)
        
        # Final classifier
        self.fc = nn.Linear(512, n_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        # First block may change channels/stride
        layers.append(ResidualBlock1D(in_channels, out_channels, stride=stride))
        # Remaining blocks keep same channels/stride=1
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock1D(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        # x shape: (batch, n_channels, n_times)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Residual layers
        x = self.layer1(x)  # shape: (batch, 64, T1)
        x = self.layer2(x)  # shape: (batch, 128, T2)
        x = self.layer3(x)  # shape: (batch, 256, T3)
        x = self.layer4(x)  # shape: (batch, 512, T4)
        
        # Global average pooling: average over time dimension
        x = self.global_pool(x)  # (batch, 512, 1)
        x = x.squeeze(-1)        # (batch, 512)
        
        # Classifier
        logits = self.fc(x)      # (batch, n_classes)
        return logits


## Building the training Loop

In [25]:
#Dataset class

class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.float()
        self.y = y.long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [26]:
# split X and Y based on subjects 
X_raw = np.load("../data/X_tqwt_wpd.npy")
y_raw = np.load("../data/y_labels.npy")

subject_ids = np.load("../data/subject_ids.npy", allow_pickle=True)
unique_subs = np.unique(subject_ids)

# Not required 
# print("Number of subjects:", len(unique_subs))
# print("Subject IDs:", unique_subs)


In [27]:
train_subs, val_subs = train_test_split(
    unique_subs,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

In [28]:
# Basically ensures that there is no leakage 

train_mask = np.isin(subject_ids, train_subs)
val_mask = np.isin(subject_ids, val_subs)

In [29]:
X = np.load("../data/X_tqwt_wpd.npy")
y = np.load("../data/y_labels.npy")

In [30]:
print("X shape:", X.shape)
print("y shape:", y.shape)
print("subject_ids shape:", subject_ids.shape)

X shape: (16749, 1140)
y shape: (16749,)
subject_ids shape: (16749,)


In [31]:
# convert to torch tensors
X1 = X[train_mask]
y1 = y[train_mask]

X_val_np = X[val_mask]
y_val_np = y[val_mask]


X_train_np = torch.tensor(X1, dtype=torch.float32)
y_train_np = torch.tensor(y1, dtype=torch.long)

X_val_np = torch.tensor(X_val_np, dtype=torch.float32)
y_val_np = torch.tensor(y_val_np, dtype=torch.long)

### Lets save the files once... No need to do this if you already have the files downloaded

In [32]:
np.save("../data/X_train.npy", X_train_np)
np.save("../data/y_train.npy", y_train_np)
np.save("../data/X_val.npy", X_val_np)
np.save("../data/y_val.npy", y_val_np)

In [33]:
# even though they were converted to tensors before saving they need to be converted back becsuse when loading them with np it becomes an array again.
X_train = np.load("../data/X_train.npy")
y_train = np.load("../data/y_train.npy")
X_val   = np.load("../data/X_val.npy")
y_val   = np.load("../data/y_val.npy")


print("Before reshaping input to DataLoader:")
print(X_train.shape)
print("Raw X_val shape:",   X_val.shape)

X_train = X_train.reshape(len(X_train), 1, -1)
X_val   = X_val.reshape(len(X_val), 1, -1)

print("after reshaping input to DataLoader:")
print(X_train.shape)
print("Raw X_val shape:",   X_val.shape)


X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)


Before reshaping input to DataLoader:
(13305, 1140)
Raw X_val shape: (3444, 1140)
after reshaping input to DataLoader:
(13305, 1, 1140)
Raw X_val shape: (3444, 1, 1140)


In [34]:
#Creating Dataloaders 

train_dataset = EEGDataset(X_train, y_train)
train_loader  = DataLoader(train_dataset, batch_size=32, shuffle=True)
                           
val_dataset = EEGDataset(X_val, y_val)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False)

In [35]:
# Safety Check
for xb, yb in train_loader:
    print("TRAIN BATCH SHAPE:", xb.shape)
    break

for xb, yb in val_loader:
    print("VAL BATCH SHAPE:", xb.shape)
    break

print("X_train shape from file:", X_train.shape)
print("X_val shape from file:", X_val.shape)

print("First element type:", type(X_train[0]))
print("First element shape:", getattr(X_train[0], 'shape', None))

TRAIN BATCH SHAPE: torch.Size([32, 1, 1140])
VAL BATCH SHAPE: torch.Size([32, 1, 1140])
X_train shape from file: torch.Size([13305, 1, 1140])
X_val shape from file: torch.Size([3444, 1, 1140])
First element type: <class 'torch.Tensor'>
First element shape: torch.Size([1, 1140])


In [36]:
#initialze the CNN

n_channels = 1      # or however many EEG channels we have
n_classes = 2        # ADHD vs Control

model = ResNet1D(n_channels=n_channels, n_classes=n_classes)

In [37]:
#letting the computer know what piece of hardware to run the training 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

## Helper functions


In [38]:
# Accuracy calculations

def accuracy_from_logits(logits, y):
    preds = torch.argmax(logits, dim=1)
    correct = (preds == y).sum().item()
    total = y.size(0)
    return correct / total

In [39]:
#Training loop

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()  # put model in "training mode"
    running_loss = 0.0
    correct = 0
    total = 0
    
    for X, y in train_loader:
        print("TRAIN LOOP SHAPE:", X.shape)
        break
        
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        logits = model(X)

        # 2. Compute loss
        loss = criterion(logits, y)

        # 3. Zero out old gradients
        optimizer.zero_grad()

        # 4. Compute gradients
        loss.backward()

        # 5. Update weights
        optimizer.step()

        # Track training accuracy & loss
        running_loss += loss.item() * X.size(0)

        _, predicted = torch.max(logits, dim=1)
        correct += (predicted == y).sum().item()
        total += y.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    return epoch_loss, epoch_acc


#only run to check if functional. and last i ran it was functional
#train_one_epoch(model, train_loader, criterion, optimizer, device)

In [40]:
#Valid loop with no gradient Updates 
# Very similar to the training loop, except this one sets the model to eval and its accompanied by other

def validate(model, val_loader, criterion, device):
    model.eval()  # evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0

    # Do NOT track gradients
    with torch.no_grad():
        for X, y in val_loader:
            print("VAL LOOP SHAPE:", X.shape)
            break
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)

            logits = model(X)
            loss = criterion(logits, y)

            running_loss += loss.item() * X.size(0)
            _, predicted = torch.max(logits, dim=1)
            correct += (predicted == y).sum().item()
            total += y.size(0)

    val_loss = running_loss / total
    val_acc = correct / total

    return val_loss, val_acc

#only run to check if functional. and last i ran it was functional
#validate(model, val_loader, criterion, device)

In [41]:
def train_model(model, train_loader, val_loader, device, epochs=20, lr=1e-3):

    history = {
        "train_loss": [],
        "train_acc":  [],
        "val_loss":   [],
        "val_acc":    []
    }

    for epoch in range(epochs):
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device
        )

        val_loss, val_acc = validate(
            model, val_loader, criterion, device
        )

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
            f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}"
        )

    return history


## Training the architectiure


In [42]:
#Training the resnet architeciure on Raw data 

# Clear GPU cache before creating model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU memory cleared. Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Try to create model on GPU, fallback to CPU if OOM
try:
    resnet_raw = ResNet1D(n_channels=X_train.shape[1], n_classes=2).to(device)
    print(f"Model created on {device}")
except RuntimeError as e:
    if "out of memory" in str(e).lower() or "cuda" in str(e).lower():
        print(f"CUDA OOM error. Falling back to CPU...")
        device = torch.device("cpu")
        torch.cuda.empty_cache()
        resnet_raw = ResNet1D(n_channels=X_train.shape[1], n_classes=2).to(device)
        print(f"Model created on CPU")
    else:
        raise

history_resnet_raw = train_model(
    model=resnet_raw,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=20,
    lr=1e-3
)

# Saving for Saliency later
os.makedirs("../models", exist_ok=True)
torch.save(resnet_raw.state_dict(), "../models/resnet_eeg.pth")
print("Model saved to ../models/resnet_eeg.pth")
# Can be loaded with the following code 
# model.load_state_dict(torch.load("resnet_eeg.pth"))

Model created on cpu
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 1/20 | Train Loss: 0.7550 Acc: 0.4561 | Val Loss: 0.7821 Acc: 0.3885
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 2/20 | Train Loss: 0.7542 Acc: 0.4561 | Val Loss: 0.7933 Acc: 0.3885
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 3/20 | Train Loss: 0.7550 Acc: 0.4561 | Val Loss: 0.7441 Acc: 0.3885
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 4/20 | Train Loss: 0.7559 Acc: 0.4561 | Val Loss: 0.8077 Acc: 0.3885
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 5/20 | Train Loss: 0.7553 Acc: 0.4560 | Val Loss: 0.7968 Acc: 0.3885
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 6/20 | Train Loss: 0.7553 Acc: 0.4561 | Val Loss: 0.7433 Acc: 0.3885
TRAIN LOOP SH

## Compute saliency 

In [43]:
def compute_saliency(model, X_batch, y_batch, device):
    model.eval()
    # Ensure model is on the correct device
    model = model.to(device)
    
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)
    X_batch.requires_grad = True  # Enable gradient wrt input

    # Forward pass
    logits = model(X_batch)
    loss = F.cross_entropy(logits, y_batch)

    # Backward pass
    loss.backward()

    # Gradient wrt input
    saliency = X_batch.grad.detach().abs()  # absolute gradient

    return saliency.cpu().numpy()


## Global saliency vector 

In [44]:
all_saliencies = []

for Xb, yb in val_loader:
    sal = compute_saliency(model, Xb, yb, device)
    all_saliencies.append(sal)

all_saliencies = np.concatenate(all_saliencies, axis=0)   # (N_val, 1, 1140)

# Average across samples and channel
global_saliency = all_saliencies.mean(axis=0).squeeze(0)  # (1140,)


In [45]:
#Normalize
global_saliency = global_saliency / global_saliency.max()

In [60]:
# Measure how many features/time points are kept vs removed
flat_mask = mask.reshape(-1)        # flatten in case it's not 1D
num_total = flat_mask.size
num_kept = flat_mask.sum()
num_removed = num_total - num_kept
percent_removed = (num_removed / num_total) * 100.0
percent_kept = 100.0 - percent_removed

print(f"Total features per sample: {num_total}")
print(f"Kept:    {num_kept:.0f}  ({percent_kept:.2f}%)")
print(f"Removed: {num_removed:.0f}  ({percent_removed:.2f}%)")


Total features per sample: 1140
Kept:    625  (54.82%)
Removed: 515  (45.18%)


## Handling the Masked data 

In [47]:
#Applying saliency mask to the dataset, actually just uses simple multiplication

# Convert tensors back to numpy so we can multiply with mask
X_train_np = X_train.cpu().numpy().reshape(len(X_train), 1140)
X_val_np   = X_val.cpu().numpy().reshape(len(X_val), 1140)

#applying the mask
X_train_masked = X_train_np * mask
X_val_masked   = X_val_np * mask

#reshaping back to conv1d format 
X_train_masked = X_train_masked.reshape(len(X_train_masked), 1, 1140)
X_val_masked   = X_val_masked.reshape(len(X_val_masked),   1, 1140)

In [48]:
# Creating the data loaders for the masked data  
y_train_masked = y_train.clone()
y_val_masked   = y_val.clone()

X_train_masked = torch.tensor(X_train_masked, dtype=torch.float32)
X_val_masked   = torch.tensor(X_val_masked,   dtype=torch.float32)

#creating the masked dataset
train_dataset_masked = EEGDataset(X_train_masked, y_train_masked)
val_dataset_masked   = EEGDataset(X_val_masked,   y_val_masked)

#dataLoaders
train_loader_masked = DataLoader(train_dataset_masked, batch_size=32, shuffle=True)
val_loader_masked   = DataLoader(val_dataset_masked,   batch_size=32, shuffle=False)


In [49]:
# converting masked data to tensors 
X_train_masked = torch.tensor(X_train_masked, dtype=torch.float32)
y_train_masked = y_train.clone()      # same labels
X_val_masked   = torch.tensor(X_val_masked,   dtype=torch.float32)
y_val_masked   = y_val.clone()

/var/folders/1_/635jq8l52k10bb3t2l5v0_400000gn/T/ipykernel_16031/922651201.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_masked = torch.tensor(X_train_masked, dtype=torch.float32)
/var/folders/1_/635jq8l52k10bb3t2l5v0_400000gn/T/ipykernel_16031/922651201.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_masked   = torch.tensor(X_val_masked,   dtype=torch.float32)


In [50]:
# creatging new dataset and dataloaders for the data 
train_dataset_masked = EEGDataset(X_train_masked, y_train_masked)
val_dataset_masked   = EEGDataset(X_val_masked,   y_val_masked)

train_loader_masked = DataLoader(train_dataset_masked, batch_size=32, shuffle=True)
val_loader_masked   = DataLoader(val_dataset_masked,   batch_size=32, shuffle=False)


In [51]:
# Checking shapes for troubleshooting purposes 
for xb, yb in train_loader_masked:
    print("MASKED TRAIN BATCH:", xb.shape)
    break

for xb, yb in val_loader_masked:
    print("MASKED VAL BATCH:", xb.shape)
    break


MASKED TRAIN BATCH: torch.Size([32, 1, 1140])
MASKED VAL BATCH: torch.Size([32, 1, 1140])


## 1Dimentional implemeentation 

In [52]:
# Eeg net 1Dimentional implemeentation 

class EEGNet1D(nn.Module):
    def __init__(
        self,
        n_classes: int = 2,
        Chans: int = 1,        # number of input channels
        Samples: int = 1140,   # length of the time series
        F1: int = 8,
        D: int = 2,
        kernel_length: int = 64,
        dropout: float = 0.25
    ):
        super().__init__()
        self.n_classes = n_classes
        self.Chans = Chans
        self.Samples = Samples

        # 1) Temporal convolution
        self.conv_temporal = nn.Conv1d(
            in_channels=Chans,
            out_channels=F1,
            kernel_size=kernel_length,
            padding=kernel_length // 2,
            bias=False
        )
        self.bn1 = nn.BatchNorm1d(F1)

        # 2) Depthwise convolution
        #    Each filter operates on its own channel (groups=F1), multiplied by D
        self.conv_depthwise = nn.Conv1d(
            in_channels=F1,
            out_channels=F1 * D,
            kernel_size=kernel_length,
            padding=kernel_length // 2,
            groups=F1,
            bias=False
        )
        self.bn2 = nn.BatchNorm1d(F1 * D)
        self.pool1 = nn.AvgPool1d(kernel_size=4)
        self.dropout1 = nn.Dropout(dropout)

        # 3) Separable convolution
        #    depthwise (groups=F1*D) + pointwise (1x1 conv)
        self.conv_separable_depth = nn.Conv1d(
            in_channels=F1 * D,
            out_channels=F1 * D,
            kernel_size=16,
            padding=16 // 2,
            groups=F1 * D,
            bias=False
        )
        self.conv_separable_point = nn.Conv1d(
            in_channels=F1 * D,
            out_channels=F1 * D * 2,  # F2 = 2 * F1 * D
            kernel_size=1,
            bias=False
        )
        self.bn3 = nn.BatchNorm1d(F1 * D * 2)
        self.pool2 = nn.AvgPool1d(kernel_size=8)
        self.dropout2 = nn.Dropout(dropout)

        # 4) Global average pooling + classifier
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(F1 * D * 2, n_classes)

        self.elu = nn.ELU()

    def forward(self, x):
        """
        x: (batch_size, 1, 1140)
        """
        # 1) Temporal conv
        x = self.conv_temporal(x)
        x = self.bn1(x)
        x = self.elu(x)

        # 2) Depthwise conv
        x = self.conv_depthwise(x)
        x = self.bn2(x)
        x = self.elu(x)
        x = self.pool1(x)
        x = self.dropout1(x)

        # 3) Separable conv
        x = self.conv_separable_depth(x)
        x = self.conv_separable_point(x)
        x = self.bn3(x)
        x = self.elu(x)
        x = self.pool2(x)
        x = self.dropout2(x)

        # 4) Global pooling + classifier
        x = self.global_pool(x)     # (batch, channels, 1)
        x = x.squeeze(-1)           # (batch, channels)
        logits = self.classifier(x) # (batch, n_classes)
        return logits

## Training both datasets 

In [61]:
# BEFORE filtering: EEGNet on raw EEG
eegnet_raw = EEGNet1D(n_classes=2, Chans=1, Samples=1140).to(device)

start_time = time.time()
history_raw = train_model(
    model=eegnet_raw,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=20,   # or 3 etc., whatever you used
    lr=1e-3
)
eegnet_raw_train_time = time.time() - start_time
print(f"EEGNet (raw) training time: {eegnet_raw_train_time:.2f} seconds")

TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 1/20 | Train Loss: 0.7013 Acc: 0.5439 | Val Loss: 0.6882 Acc: 0.6115
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 2/20 | Train Loss: 0.7014 Acc: 0.5439 | Val Loss: 0.6828 Acc: 0.6115
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 3/20 | Train Loss: 0.7011 Acc: 0.5440 | Val Loss: 0.6858 Acc: 0.6115
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 4/20 | Train Loss: 0.7030 Acc: 0.5439 | Val Loss: 0.6853 Acc: 0.6115
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 5/20 | Train Loss: 0.7019 Acc: 0.5439 | Val Loss: 0.6862 Acc: 0.6115
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 6/20 | Train Loss: 0.7025 Acc: 0.5441 | Val Loss: 0.6901 Acc: 0.6115
TRAIN LOOP SHAPE: torch.Size([32, 

In [62]:
# AFTER filtering: EEGNet on masked EEG
eegnet_masked = EEGNet1D(n_classes=2, Chans=1, Samples=1140).to(device)

start_time = time.time()
history_masked = train_model(
    model=eegnet_masked,
    train_loader=train_loader_masked,
    val_loader=val_loader_masked,
    device=device,
    epochs=20,   # same as above
    lr=1e-3
)
eegnet_masked_train_time = time.time() - start_time
print(f"EEGNet (masked) training time: {eegnet_masked_train_time:.2f} seconds")


TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 1/20 | Train Loss: 0.6960 Acc: 0.4732 | Val Loss: 0.7013 Acc: 0.4120
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 2/20 | Train Loss: 0.6963 Acc: 0.4744 | Val Loss: 0.7023 Acc: 0.4164
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 3/20 | Train Loss: 0.6960 Acc: 0.4702 | Val Loss: 0.7020 Acc: 0.4126
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 4/20 | Train Loss: 0.6959 Acc: 0.4741 | Val Loss: 0.7012 Acc: 0.4117
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 5/20 | Train Loss: 0.6959 Acc: 0.4712 | Val Loss: 0.7017 Acc: 0.4184
TRAIN LOOP SHAPE: torch.Size([32, 1, 1140])
VAL LOOP SHAPE: torch.Size([32, 1, 1140])
Epoch 6/20 | Train Loss: 0.6956 Acc: 0.4767 | Val Loss: 0.7006 Acc: 0.4262
TRAIN LOOP SHAPE: torch.Size([32, 

## Simple evaluation

In [55]:
resnet_loaded = ResNet1D(n_channels=1, n_classes=2).to(device)
resnet_loaded.load_state_dict(torch.load("../models/resnet_eeg.pth", map_location=device))
resnet_loaded.eval()

ResNet1D(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): ResidualBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNo

In [56]:
# Evaluate with metrics 
def evaluate_model(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X, y in data_loader:      
            X = X.to(device)
            y = y.to(device)

            logits = model(X)
            preds = torch.argmax(logits, dim=1)

            all_preds.append(preds.cpu())
            all_labels.append(y.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    # ADHD vs Control: binary classification.
    # If it ever becomes multi-class, change average="macro".
    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average="binary"
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [57]:
# Evaluate all three models and record inference time

start = time.time()
metrics_resnet = evaluate_model(resnet_loaded, val_loader, device)
resnet_infer_time = time.time() - start

start = time.time()
metrics_eegnet_raw = evaluate_model(eegnet_raw, val_loader, device)
eegnet_raw_infer_time = time.time() - start

start = time.time()
metrics_eegnet_masked = evaluate_model(eegnet_masked, val_loader_masked, device)
eegnet_masked_infer_time = time.time() - start


/opt/anaconda3/envs/eeg-ml/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [59]:
def print_metrics(name, metrics, infer_time):
    print(f"{name}:")
    print(f"Accuracy:  {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall:    {metrics['recall']:.4f}")
    print(f"F1 score:  {metrics['f1']:.4f}")
    print(f"Inference time: {infer_time:.4f} seconds")
    print()

print_metrics("ResNet (raw)", metrics_resnet, resnet_infer_time)
print_metrics("EEGNet (raw)", metrics_eegnet_raw, eegnet_raw_infer_time)
print_metrics("EEGNet (masked)", metrics_eegnet_masked, eegnet_masked_infer_time)


ResNet (raw):
Accuracy:  0.3885
Precision: 0.0000
Recall:    0.0000
F1 score:  0.0000
Inference time: 58.3219 seconds

EEGNet (raw):
Accuracy:  0.6353
Precision: 0.6298
Recall:    0.9791
F1 score:  0.7665
Inference time: 10.3454 seconds

EEGNet (masked):
Accuracy:  0.4271
Precision: 0.6291
Recall:    0.1538
F1 score:  0.2472
Inference time: 10.3429 seconds

